Assignment 3

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation,Dense, Flatten, MaxPooling2D, Dropout, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Number of GPUs Available: ",len(physical_devices))
tf.config.experimental.set_memory_growth(physical_devices[0],True)

In [ ]:
# Load CIFAR Data set.
cifar10 = tf.keras.datasets.cifar10 # tf.keras.datasets.cifar10.load_data()
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

print(train_images.shape)
print(test_images.shape)
print(train_labels.shape)
print(train_images[0].shape)
#show the first image.
plt.imshow(train_images[0])

In [ ]:
data = tf.data.Dataset.from_tensor_slices(
    (train_images.reshape([-1, 32, 32, 3]).astype(np.float32) / 255, train_labels.reshape(-1,).astype(np.int32)))
data = data.shuffle(buffer_size=50000).batch(128).repeat()

test_data = tf.data.Dataset.from_tensor_slices(
    (test_images.reshape([-1, 32, 32, 3]).astype(np.float32) / 255, test_labels.reshape(-1,).astype(np.int32))).batch(10000)

num_classes = 10

In [ ]:
def Genreate_CNN_Arch(conv_layers,dense_layers,num_classes,lastlayeractivation='softmax'):
  model = Sequential()
  for idx in range(len(conv_layers)):
    layer = conv_layers[idx]
    if idx == 0:
      model.add(Conv2D(layer[0],(layer[1],layer[1]),strides=(layer[2],layer[2]),padding=layer[3],activation=layer[4],input_shape=(32,32,3))) # input_shape=(32,32,3)
    else:
      model.add(Conv2D(layer[0],(layer[1],layer[1]),strides=(layer[2],layer[2]),padding=layer[3],activation=layer[4]))
    if (len(layer) > 5 ): # Maxpooling 
      model.add(MaxPooling2D(pool_size=(layer[5],layer[5]),strides=(layer[6],layer[6]),padding=layer[7]))

  model.add(Flatten())

  # Add Denselayer
  for idx in range(len(dense_layers)):
    layer = dense_layers[idx]
    #print("layer: ",layer)
    model.add(Dense(units=layer[0],activation=layer[1]))

  #output layer.
  #print("lastlayeractivation: ",lastlayeractivation)
  model.add(Dense(num_classes,activation=lastlayeractivation ))

  return model

In [ ]:
def train_models(model,data,train_steps,parameters,optimizer,loss_func):
  for step ,(img_batch,lbl_batch) in enumerate(data):
    #print("step: ",step)
    if step > train_steps:
      break
  
    with tf.GradientTape() as tape:
      logits = model(img_batch)
      xent = loss_func(lbl_batch, logits)

    grads = tape.gradient(xent,parameters)

    optimizer.apply_gradients(zip(grads,parameters))

    if not step % 100:
      predict = tf.argmax(logits,axis=1,output_type=tf.int32)
      accuracy = tf.reduce_mean(tf.cast(tf.equal(predict,lbl_batch),tf.float32))
      print("Loss: {} Accuracy: {}".format(xent,accuracy))

In [ ]:
def get_test_accuracy(model,test_data):
  test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

  for img_batch, lbl_batch in test_data:
    test_accuracy(lbl_batch,model(img_batch))

  print("Test Accuracy: ",test_accuracy.result())

In [26]:
train_steps = 20000
optimizer = tf.keras.optimizers.Adam(0.001)
loss_func = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

# number_of_filters, filter_size, convstride, convpadding, activation, max_pool_size, maxpoolstrides, maxpoolpadding.
conv_layers = [[64,3,2,"valid",'relu',1,2,"valid"],[64,3,1,"valid",'relu',2,1,"valid"],[64,1,2,"valid",'relu',2,1,"valid"]]
#number_of_units, activation function.
dense_layers = [[1000,'relu'],[1000,'relu']]

model = Genreate_CNN_Arch(conv_layers,dense_layers,10)
parameters = model.trainable_variables
model.summary()
train_models(model,data,train_steps,parameters,optimizer,loss_func)
get_test_accuracy(model,test_data)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 15, 15, 64)        1792      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 6, 6, 64)          36928     
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 3, 3, 64)          4160      
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 2, 2, 64)          0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 256)             

Architecture2: Filter size = 1 * 1 , 2 conv layers with maxpooling, Dense layer = 100 and last layer activation as relu.

In [27]:
train_steps = 2000
optimizer = tf.keras.optimizers.Adam(0.001)
loss_func = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

# number_of_filters, filter_size, convstride, convpadding, activation, max_pool_size, maxpoolstrides, maxpoolpadding.
conv_layers = [[64,1,1,"valid",'relu',1,2,"valid"],[64,1,1,"valid",'relu',2,1,"valid"]]
#number_of_units, activation function.
dense_layers = [[1000,'relu'],[1000,'relu']]

model = Genreate_CNN_Arch(conv_layers,dense_layers,10)
parameters = model.trainable_variables
model.summary()
train_models(model,data,train_steps,parameters,optimizer,loss_func)
get_test_accuracy(model,test_data)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_35 (Conv2D)           (None, 32, 32, 64)        256       
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 16, 16, 64)        4160      
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 15, 15, 64)        0         
_________________________________________________________________
flatten_11 (Flatten)         (None, 14400)             0         
_________________________________________________________________
dense_28 (Dense)             (None, 1000)              14401000  
_________________________________________________________________
dense_29 (Dense)             (None, 1000)            

Architecture 3 : filter size = image size ( 28 * 28 ) for both conv layers. It will lead to failing of model generation since output will very small for next layer to apply the filter.

In [28]:
train_steps = 2000
optimizer = tf.keras.optimizers.Adam(0.001)
loss_func = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

# number_of_filters, filter_size, convstride, convpadding, activation, max_pool_size, maxpoolstrides, maxpoolpadding.
conv_layers = [[64,28,1,"valid",'relu',1,2,"valid"],[64,28,1,"valid",'relu',2,1,"valid"]]
#number_of_units, activation function.
dense_layers = [[1000,'relu']]

model = Genreate_CNN_Arch(conv_layers,dense_layers,10)
#parameter initialization
parameters = model.trainable_variables
model.summary()
train_models_new(model,data_mf,train_steps,parameters,optimizer,loss_func)
get_test_accuracy(model,test_data_mf)

ValueError: ignored

In [29]:
train_steps = 10000
optimizer = tf.keras.optimizers.Adam(0.01)
loss_func = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

# number_of_filters, filter_size, convstride, convpadding, activation, max_pool_size, maxpoolstrides, maxpoolpadding.
conv_layers = [[16,2,1,"valid",'relu',1,2,"valid"],[32,2,1,"valid",'relu',2,1,"valid"]]
#number_of_units, activation function.
dense_layers = [[64,'relu']]

model = Genreate_CNN_Arch(conv_layers,dense_layers,10)
parameters = model.trainable_variables
model.summary()
train_models(model,data,train_steps,parameters,optimizer,loss_func)
get_test_accuracy(model,test_data)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 31, 31, 16)        208       
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 16, 16, 16)        0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 15, 15, 32)        2080      
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 6272)              0         
_________________________________________________________________
dense_31 (Dense)             (None, 64)                401472    
_________________________________________________________________
dense_32 (Dense)             (None, 10)              